<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [2]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [3]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"

print(base_path)

./data/PhilipsHueofficial/


In [8]:


def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    if "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col):
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )


    try:
        return int(col.replace(" ", "").replace(".", "").replace(",", "."))
    except:
        return 0


def trata_comments(col):
    # print(col)
    # print(type(col))

    if type(col) == int or type(col) == float:
        return col
    
    value = 0

    try:
        value = int(col)
    except:
        pass


    if 'Transcrição' in col:
        return 0
    
    col = col.replace("\xa0", "")

   
    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        try:
               value = int(
                float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
            )
        except:
            value =  0

    return int(col.replace(" ", "").replace(".", "").replace(",", ".")) if value != 0 else 0


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data




trata videos

In [9]:

videos = pd.read_csv(base_path + "videos.csv")

display(videos)
display(videos.columns)

,title,views,likes,date,comments_amount,channel_name,subscribers
0,Live with Hue – den ultimata smarta belysninge...,366 visualizações,0,28 de set. de 2023,NaN,Philips Hue,19 mil inscritos
1,Philips Hue Secure: Bright home security,362 visualizações,0,26 de set. de 2023,NaN,Philips Hue,19 mil inscritos
2,Philips Hue Secure: Bright home security,356 visualizações,0,26 de set. de 2023,NaN,Philips Hue,19 mil inscritos
3,Philips Hue Secure: Bright home security,151 visualizações,0,26 de set. de 2023,NaN,Philips Hue,19 mil inscritos
4,Philips Hue Secure: Bright home security,116 visualizações,0,26 de set. de 2023,NaN,Philips Hue,19 mil inscritos
5,interruptor inteligente 4x4 com 6 botões zigbe...,visualizações,2,4 de out. de 2023,Transcrição,EKAZA,"2,1 mil inscritos"
6,Como conectar sua câmera mini dome no DVR usan...,22 visualizações,3,2 de out. de 2023,Transcrição,EKAZA,"2,1 mil inscritos"
7,Interruptor INTELIGENTE 4x4 4 botões + 2 Tomad...,51 visualizações,4,29 de set. de 2023,Transcrição,EKAZA,"2,1 mil inscritos"
8,Como conectar o nutri alimentador sem câmera n...,45 visualizações,4,28 de set. de 2023,2,EKAZA,"2,1 mil inscritos"
9,Você PRECISA ver esse vídeo antes de comprar u...,311 visualizações,16,26 de set. de 2023,7,EKAZA,"2,1 mil inscritos"


Index(['title', 'views', 'likes', 'date', 'comments_amount', 'channel_name',
       'subscribers'],
      dtype='object')

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:


videos.assign(
    subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="RECENT",
).to_parquet(base_path + "videos.parquet")


# Trata Top videos

In [11]:
import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")

display(top_videos.head())

,title,views,likes,date,comments_amount,channel_name
0,Personalized smart lighting for your entire home,17 mi de visualizações,0,29 de set. de 2020,NaN,Philips Hue
1,Scopri l'illuminazione intelligente di Philips...,11 mi de visualizações,25,22 de dez. de 2020,2.0,Philips Hue
2,L'éclairage connecté Philips Hue suivant vos a...,"9,1 mi de visualizações",0,12 de out. de 2020,NaN,Philips Hue
3,Philips Hue - osvětlete svůj domov chytře,"6,6 mi de visualizações",15,3 de nov. de 2020,4.0,Philips Hue
4,Personlig smart belysning för hela ditt hem,"5,4 mi de visualizações",0,18 de jan. de 2021,NaN,Philips Hue


In [12]:

top_videos.assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="TOP",
).to_parquet(base_path + "top_videos.parquet")


In [13]:


top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")



In [14]:

# # Trata lifetime


df_lifetime = pd.read_csv(base_path + "lifetime.csv")


df_lifetime.head()



def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days



df_lifetime = (
    df_lifetime.assign(
        date_joined=lambda df_: df_["date_joined"].apply(trata_data),
        total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
        total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
        days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
    )
)#.to_csv(base_path + "lifetime_clean.csv", index=False)




In [15]:
df_lifetime

,channel_title,date_joined,total_views,total_subs,total_videos,days_since_joined
0,Philips Hue,2018-07-13,84793960,19000,393,1909
1,EKAZA,2018-11-01,300238,2100,64,1798
2,Positivo Casa Inteligente,2019-06-11,6911336,6610,49,1576
3,Elsys,2012-04-19,7437940,36900,264,4185
4,Xiaomi Brasil,2015-05-11,108447,45100,67,3068


In [16]:
lifetime_csv = pd.DataFrame()

lifetime_csv['Channel Id'] = np.arange(5)

lifetime_csv[' Channel Name'] = df_lifetime['channel_title']
lifetime_csv[' Days on YouTube'] = df_lifetime['days_since_joined']
lifetime_csv[' Total Views'] = df_lifetime['total_views']
lifetime_csv[' Views per Day'] = lifetime_csv[' Total Views'] / lifetime_csv[' Days on YouTube']
lifetime_csv[' Total Subscribers'] = df_lifetime['total_subs']
lifetime_csv[' Subscribers per Day'] = lifetime_csv[' Total Subscribers'] / lifetime_csv[' Days on YouTube']
lifetime_csv[' Total Uploads'] = df_lifetime['total_videos']
lifetime_csv[' Uploads per Month'] = lifetime_csv[' Total Uploads'] / (lifetime_csv[' Days on YouTube'] / 30)
lifetime_csv[''] = np.arange(5)

lifetime_csv.to_csv(base_path + "Lifetime.csv", index=False, decimal=',')

In [17]:
# # join


videos = pd.read_parquet(base_path + "videos.parquet")
top_videos = pd.read_parquet(base_path + "top_videos.parquet")

videos = (
    videos.assign(
        title_canal = lambda d: d['channel_name'] + ' - ' + d['title']
    )
)

top_videos = (
    top_videos.assign(
        title_canal = lambda d: d['channel_name'] + ' - ' + d['title']
    )
)
    


# videos.sort_values("channel_name").to_csv(base_path + "videos_clean.csv", index=False, sep=";")
# top_videos.sort_values("channel_name").to_csv(base_path + "top_videos_clean.csv", index=False, sep=";")


# # join the two dataframes

# videos = pd.concat([videos, top_videos])


# videos_sorted = videos.sort_values("channel_name")

# videos_sorted["subscribers"] = videos_sorted["subscribers"].fillna(method="ffill")


# videos_sorted = videos_sorted.sort_values(["channel_name", "category", "date"])


# videos_sorted.to_parquet(base_path + "videos_sorted.parquet")


# (pd.read_parquet(base_path + "videos_sorted.parquet")
#  .assign(title_canal = lambda d: d['channel_name'] + ' - ' + d['title'])
#  .to_csv(
#     base_path + "videos_sorted.csv", index=False, sep=';'
# ))

In [18]:
videos_aux = pd.DataFrame()


videos_aux['Channel Id'] = np.arange(1, len(videos) + 1)
videos_aux[' Channel Name'] = videos['channel_name']
videos_aux[' Video Id'] = np.arange(1, len(videos) + 1)
videos_aux[' VideoUrl'] = np.arange(1, len(videos) + 1)
videos_aux[' Title'] = videos['title_canal']
videos_aux[' Published Date'] = videos['date']
videos_aux[' Views'] = videos['views']
videos_aux[' Likes'] = videos['likes']
videos_aux[' Dislikes'] = np.arange(1, len(videos) + 1)
videos_aux[' Comments'] = videos['comments_amount']
videos_aux[' Engagement %'] = np.arange(1, len(videos) + 1)

In [19]:
videos_aux.to_csv(base_path + "RecentUploads.csv", index=False)

In [20]:
top_videos.head()

,title,views,likes,date,comments_amount,channel_name,category,title_canal
0,Personalized smart lighting for your entire home,17000000,0,2020-09-29,NaN,Philips Hue,TOP,Philips Hue - Personalized smart lighting for ...
1,Scopri l'illuminazione intelligente di Philips...,11000000,25,2020-12-22,2.0,Philips Hue,TOP,Philips Hue - Scopri l'illuminazione intellige...
2,L'éclairage connecté Philips Hue suivant vos a...,9100000,0,2020-10-12,NaN,Philips Hue,TOP,Philips Hue - L'éclairage connecté Philips Hue...
3,Philips Hue - osvětlete svůj domov chytře,6600000,15,2020-11-03,4.0,Philips Hue,TOP,Philips Hue - Philips Hue - osvětlete svůj dom...
4,Personlig smart belysning för hela ditt hem,5400000,0,2021-01-18,NaN,Philips Hue,TOP,Philips Hue - Personlig smart belysning för he...


In [21]:
videos_aux = pd.DataFrame()


videos_aux['Channel Id'] = np.arange(1, len(videos) + 1)
videos_aux[' Channel Name'] = top_videos['channel_name']
videos_aux[' Video Id'] = np.arange(1, len(videos) + 1)
videos_aux[' VideoUrl'] = np.arange(1, len(videos) + 1)
videos_aux[' Title'] = top_videos['title_canal']
videos_aux[' Published Date'] = top_videos['date']
videos_aux[' Views'] = top_videos['views']
videos_aux[' Likes'] = top_videos['likes']
videos_aux[' Dislikes'] = np.arange(1, len(videos) + 1)
videos_aux[' Comments'] = top_videos['comments_amount']
videos_aux[' Engagement %'] = np.arange(1, len(videos) + 1)

In [22]:
videos_aux.to_csv(base_path + "TopUploads.csv", index=False)

last 30 days

In [23]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), 
    subs=lambda x: fix_views(x["subs"])
)

# Replace '---' with NaN in the 'subs' column
last_30days['subs'] = last_30days['subs'].replace('--', np.nan)



In [24]:
last_30days = last_30days.astype({'views': 'float64', 'subs': 'float64'})


In [25]:
last_30_days_aux = pd.DataFrame()



last_30_days_aux['Channel Id'] = np.arange(5)
last_30_days_aux[' Channel Name'] = df_lifetime['channel_title']
last_30_days_aux['  Views Last 30 Days'] = last_30days['views']
last_30_days_aux[' Views per Day'] = last_30_days_aux['  Views Last 30 Days']/30
last_30_days_aux[' Total Subscribers Last 30 Days'] = last_30days['subs']
last_30_days_aux[' Subscribers per Day'] = last_30_days_aux[' Total Subscribers Last 30 Days']/30
last_30_days_aux[' Total Uploads Last 30 Days'] = ["ATUALIZAR", "ATUALIZAR", "ATUALIZAR", "ATUALIZAR", "ATUALIZAR"]
last_30_days_aux[' Uploads per Week'] = np.arange(5)
last_30_days_aux[''] = np.arange(5)








In [26]:
last_30_days_aux

,Channel Id,Channel Name,Views Last 30 Days,Views per Day,Total Subscribers Last 30 Days,Subscribers per Day,Total Uploads Last 30 Days,Uploads per Week,
0,0,Philips Hue,175264.0,5842.133333,200.0,6.666667,ATUALIZAR,0,0
1,1,EKAZA,25058.0,835.266667,140.0,4.666667,ATUALIZAR,1,1
2,2,Positivo Casa Inteligente,13611.0,453.700000,150.0,5.000000,ATUALIZAR,2,2
3,3,Elsys,0.0,0.000000,0.0,0.000000,ATUALIZAR,3,3
4,4,Xiaomi Brasil,1376.0,45.866667,NaN,NaN,ATUALIZAR,4,4


In [27]:
last_30_days_aux.to_csv(base_path + "Last30Days.csv", index=False, decimal=',')

In [28]:

# last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)